In [1]:
import json
import jieba
import os
import re
import codecs
import binascii
import time

In [2]:
jieba.set_dictionary('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/big/cute.txt')
jieba.load_userdict('./Workspace/sentiment/0616/big/jieba356726.txt')
jieba.load_userdict('./Workspace/sentiment/0616/food/fooddict2027.txt')
jieba.load_userdict('./Workspace/sentiment/0616/menu/menu50806_new.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negativewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/positivewords.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/negative.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/more.txt')
jieba.load_userdict('./Workspace/sentiment/0616/sentiment/question.txt')
jieba.load_userdict('./Workspace/sentiment/0616/stop/stopword2292.txt')
jieba.enable_parallel(6)

Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
DEBUG:jieba:Building prefix dict from /Users/fan/anaconda/bin/Workspace/sentiment/0616/big/jieba356726.txt ...
Loading model from cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
DEBUG:jieba:Loading model from cache /var/folders/ws/qq63c39d43l_20sybqcwcf900000gn/T/jieba.u60b07a305259704c4cd827282b04b44e.cache
Loading model cost 0.398 seconds.
DEBUG:jieba:Loading model cost 0.398 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [3]:
def deleteBadWords(StrIn):
    Str_BadWords = u'延伸閱讀|連絡方式|電話預約|電話|營業時間|週一|週二|週三|週四|週五|週六|週日|周一|周二|周三|周四|周五|周六|\
                    |周日|假日|公休|平日|地址|粉絲團|星期|禮拜|時間限制|您或許對這些文章有興趣|造訪日期|全年無休|最後點餐|營業|AM|PM|上一篇|下一篇|\
                    |分享此文|您可能喜歡的文章|懶人包|臉書|Facebook|facebook|FB|fb|全世界便宜住宿看這兒|下載愛食記App隨時觀看|按個讚啦|喜歡我的分享嗎|\
                    |瘋台灣民宿網|官方網站|瀏覽人次|最新消息|餐廳名稱|消費時間|無圖文版|網誌|Postedonby|新鮮關注回聲|Christabelle的藝想世界部落格由製作|\
                    |也許對這些文章也有興趣|發表迴響|電子郵件|必要欄位標記|電子郵件|個人網站|輸入圖片顯示文字好證明你不是機器人|站內搜尋分類|最新動態|\
                    |並不會被公開|你的位址 |迴響名稱|用餐日期|留言|載入中|文章文章|粉絲頁|發表|每人平均價位|按個讚|推薦你閱讀|Instagram|instagram|\
                    |美食地圖|版權所有|網友回應|歡迎加入|標籤|著作權聲明|非經授權|不得轉載'
    strClean = re.sub(Str_BadWords,'',StrIn)
    return strClean

def EnglishFullToHalf(StrIn):
    def transform(ele):
        alphabetInt = int(repr(ele.group('number'))[4:8],16)
        transAlphabeInt = alphabetInt - 65248
        return binascii.a2b_hex(hex(transAlphabeInt)[2:4])
    pattern = re.sub(u'(?P<number>[\uff21-\uff3a\uff41-\uff5a])', transform, StrIn)
    return pattern

def setBreakPoint(StrIn):
    pattern = u'[,，.。~～!！?？；]+'
    result = re.sub(pattern, ' ', StrIn)
    return result

def retain_English_Chinese_Arabic_numerals(StrIn):
    Str_English_Chinese = u'([^ 0-9a-zA-Zａ-ｚＡ-Ｚ\u4E00-\u9FCC]+)'
    #Str_English_Chinese = u'([^a-z^A-Z^ａ-ｚ^Ａ-Ｚ^^0-9^０-９^\u3105-\u3129^\u4E00-\u9FCC]+)'
    #\u3105-\u3129為所有注音符號 
    #\u4E00-\u9FCC為所有中文
    strClean = re.sub(Str_English_Chinese,'',StrIn)
    return strClean

def removeSentimentInAds(StrIn):
    pattern = u'.*(喜歡|推薦|喜愛).{0,6}(文章|本文|介紹)'
    def sub(match):
        string = match.group(0)
        type1 = match.group(1) if match.group(1) else ''
        r = string.replace(type1,'')
        return r
    result = re.sub(pattern, sub, StrIn)
    return result

## 讀資料、切詞

In [2]:
with open('./Workspace/ifoodieBlogDic20160619Update5PixnetSmallEnglish.json', 'r') as f:
    ifoodie = json.load(f)
print len(ifoodie)

72838


In [3]:
mobile01 = []
for filename in os.listdir('./Workspace/data/mobile01'):
    if 'article' in filename:
        with codecs.open('./Workspace/data/mobile01/%s' %filename, 'r', 'utf-8') as f:
            for article in f.readlines():
                mobile01.append(article.lower())
print len(mobile01)

62253


In [4]:
openrice = []
path = '/Users/fan/Downloads/openriceBlog-1/'
for fname in os.listdir(path):
    if 'openrice' in fname:
        with codecs.open(path+fname, 'r', 'utf-8') as f:
            for item in f.readlines():
                openrice.append(item.strip().lower())
print len(openrice)

61601


In [7]:
articleMerged = [ele['article'] for ele in ifoodie.values()]
articleMerged.extend(mobile01)
articleMerged.extend(openrice)
print len(articleMerged)

196692


In [ ]:
with open('./Workspace/data/article_full.txt', 'a') as f:
    for art in articleMerged:
        art = EnglishFullToHalf(art)
        art = setBreakPoint(art)
        art = art.lower()
        art = art.split()
        for s in art:
            f.write(s.encode('utf-8')+'\n')

In [ ]:
with open('./Workspace/data/article_for_mongo.txt', 'a') as f:
    for art in articleMerged:
        f.write(art.encode('utf-8')+'\n')

In [ ]:
import json
from pymongo import MongoClient
client = MongoClient('192.168.196.48', 27017)
db = client['userdata']
with open('./Workspace/data/article_for_mongo.txt', 'r') as f:
    map(lambda s: db.article.insert_one({"article":s.strip()}),f.readlines())

In [8]:
start = time.time()
with open('./Workspace/data/jieba_cut_by_sentence_big_v2.txt', 'a') as f:
    for a in articleMerged:
        a = EnglishFullToHalf(a)
        a = setBreakPoint(a)
        a = a.split()
        for s in a:
            sentence = ' '.join(list(jieba.cut(s)))
            f.write(sentence.encode('utf-8') + '\n')
print u"運行時間：%f秒" %(time.time()-start)

運行時間：2772.002258秒


## 切詞&丟回blog

In [ ]:
stopwords = []
with codecs.open('./Workspace/sentiment/0616/stop/stopword2292.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        stopwords.append(w.split()[0])
print len(stopwords)

In [ ]:
sentiment = []
with codecs.open('./expand/negativeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        sentiment.append(w.split()[0])
with codecs.open('./expand/positiveWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        sentiment.append(w.split()[0])
with codecs.open('./expand/inverseWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        sentiment.append(w.split()[0])
with codecs.open('./expand/degreeWordMerge.txt', 'r', 'utf-8') as f:
    for w in f.readlines():
        sentiment.append(w.split()[0])

In [ ]:
final = []
for w in stopwords:
    if w not in sentiment:
        final.append(w)
print len(final)

In [ ]:
print '|'.join(final)

In [ ]:
for ele in ifoodie:
    art = deleteBadWords(ifoodie[ele]['article'])
    art = EnglishFullToHalf(art)
    art = setBreakPoint(art).split()
    art_cut = [list(jieba.cut(sentence)) for sentence in art]
    article = []
    for s_cut in art_cut:
        temp = retain_English_Chinese_Arabic_numerals(' '.join(s_cut))
        s_result = [w for w in temp.split() if w not in final]
        article.append(s_result)
    ifoodie[ele]['article'] = article
with open('./Workspace/data/blog_perfect_cut.json', 'w') as f:
    json.dump(ifoodie, f)